## Generating realistic synthetic data.

In [3]:
# Generating the dataset

# Importing the necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Simulate 60 days (2 months) of data
mitai = ['Gulab Jamun', 'Ras Malai', 'Badam Pak', 'Barfi', 'Ladoo', 'Kaju Katri']  # Indian Sweets
data = []
start_date = datetime(2024, 11, 1)  # Start Date: 1st November 2024

for i in range(1, 61):
    date = start_date + timedelta(days=i)
    for _ in range(random.randint(3, 6)):  # 3–6 mitai sales per day
        data.append({
            'id': random.randint(100, 1000),
            'Mitai_Sold': random.choice(mitai),
            'Date': date.date().isoformat(),
            'Amount': random.randint(100, 4500),  # amount paid in ksh
            'Last_Updated': (date + timedelta(hours=random.randint(0, 23),
                                              minutes=random.randint(0, 59))).isoformat()
        })
        
df = pd.DataFrame(data)
df.to_csv('mitai_sales_data.csv', index=False)

# Display the first 5 rows
df.head()

,id,Mitai_Sold,Date,Amount,Last_Updated
0,636,Ras Malai,2024-11-02,376,2024-11-02T05:31:00
1,405,Barfi,2024-11-02,1384,2024-11-02T04:28:00
2,332,Badam Pak,2024-11-02,4000,2024-11-02T06:12:00
3,291,Barfi,2024-11-02,1002,2024-11-02T13:17:00
4,854,Kaju Katri,2024-11-03,2202,2024-11-03T01:55:00


## Section 1: Full Extraction

•
Load the entire dataset

•
Display basic stats (rows, columns, sample)

•
Print a message like: “Extracted X rows fully.”

In [11]:
# Implementing FULL EXTRACTION

# Loading the entire Dataset
import pandas as pd 
df_full = pd.read_csv("mitai_sales_data.csv", parse_dates=["Last_Updated"])

# Displaying basic statistics
print(f"Extracted {len(df_full)} rows fully.")

num_rows, num_columns = df_full.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
      
df_full.head()

Extracted 282 rows fully.
Number of rows: 282
Number of columns: 5


,id,Mitai_Sold,Date,Amount,Last_Updated
0,636,Ras Malai,2024-11-02,376,2024-11-02 05:31:00
1,405,Barfi,2024-11-02,1384,2024-11-02 04:28:00
2,332,Badam Pak,2024-11-02,4000,2024-11-02 06:12:00
3,291,Barfi,2024-11-02,1002,2024-11-02 13:17:00
4,854,Kaju Katri,2024-11-03,2202,2024-11-03 01:55:00


## Section 2: Incremental Extraction
•
Simulate a last extraction time in last_extraction.txt

•
Extract only new or updated records

•
Print: “Extracted Y rows incrementally since last check.”

In [2]:
# Setting initial last extraction time (halfway through the data range)
with open("last_extraction.txt", "w") as f:
    f.write("2024-12-01 12:00:00")  # last extraction time: 1st December 2024 at 12PM

In [3]:
# Implementing INCREMENTAL EXTRACTION
import pandas as pd 

with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()

df = pd.read_csv("mitai_sales_data.csv", parse_dates=["Last_Updated"])
last_extraction_time = pd.to_datetime(last_extraction)

# Extracing only new or updated records.
df_incremental = df[df['Last_Updated'] > last_extraction_time]

print(f"Extracted {len(df_incremental)} rows incrementally since last checked at {last_extraction}.")
df_incremental.head()

Extracted 139 rows incrementally since last checked at 2024-12-01 12:00:00.


,id,Mitai_Sold,Date,Amount,Last_Updated
139,366,Barfi,2024-12-01,2776,2024-12-01 18:44:00
142,520,Barfi,2024-12-01,2488,2024-12-01 12:09:00
145,675,Gulab Jamun,2024-12-02,3890,2024-12-02 06:57:00
146,121,Ras Malai,2024-12-02,3838,2024-12-02 03:19:00
147,814,Ladoo,2024-12-02,4084,2024-12-02 02:38:00


## Section 3: Save New Timestamp

•
After successful incremental extraction, update the last_extraction.txt

In [4]:
# Getting the most recent update
new_checkpoint = df['Last_Updated'].max()

# Save it
with open("last_extraction.txt", "w") as f:
    f.write(new_checkpoint.isoformat())
print(f"Updated last_extraction.txt to {new_checkpoint}")

Updated last_extraction.txt to 2024-12-31 22:55:00


## Section 4: Transform Full Data

Apply at least 3 transformations to both datasets ('transformed_full.csv', 'transformed_incremental.csv'). Examples:

- Cleaning: Handle missing values, remove duplicates.

- Enrichment: Add calculated columns (e.g., total_price = quantity * unit_price).

- Structural: Convert data types, standardize date formats.

- Filtering: Remove irrelevant columns/rows.

- Categorization: Bin numerical values (e.g., age groups).

In [ ]:
import pandas as pd

# To check the structure of the dataframe
# print(df_full.info())

# Converting 'Date' and 'Last_Updated' to datetime
df_full['Date'] = pd.to_datetime(df_full['Date'])
df_full['Last_Updated'] = pd.to_datetime(df_full['Last_Updated'])

# Checking for missing values
m_v = df_full.isnull().sum()
print(f"Number of missing values: {m_v}")

# Incase of missing values in the 'Amount' column, fill them with the median
# df_full['Amount'].fillna(df_full['Amount'].median(), inplace=True)

# Checking for duplicates
d_v = df_full.duplicated().sum()
print(f"Number of duplicates: {d_v}")

# Incase of duplicates, drop them
# df_full.drop_duplicates(inplace=True)

# Categorizing the 'Amount'
bins = [0, 1000, 2000, 3000, 4000, 5000]
labels = ['0-1000', '1001-2000', '2001-3000', '3001-4000', '4001-5000']
df_full['Amount_Category'] = pd.cut(df_full['Amount'], bins=bins, labels=labels, right=False)

df_full.to_csv('transformed_full.csv', index=False)


Number of missing values: id                 0
Mitai_Sold         0
Date               0
Amount             0
Last_Updated       0
Amount_Category    0
dtype: int64
Number of duplicates: 0


## Section 5: Transform Incremental Data

In [4]:
import pandas as pd

# To check the structure of the dataframe
# print(df_incremental.info())

# Converting 'Date' and 'Last_Updated' to datetime
df_incremental['Date'] = pd.to_datetime(df_incremental['Date'])
df_incremental['Last_Updated'] = pd.to_datetime(df_incremental['Last_Updated'])

# Checking for missing values
m_v = df_incremental.isnull().sum()
print(f"Number of missing values: {m_v}")

# Incase of missing values in the 'Amount' column, fill them with the median
# df_incremental['Amount'].fillna(df_incremental['Amount'].median(), inplace=True)

# Checking for duplicates
d_v = df_incremental.duplicated().sum()
print(f"Number of duplicates: {d_v}")

# Incase of duplicates, drop them
# df_incremental.drop_duplicates(inplace=True)

# Categorizing the 'Amount'
bins = [0, 1000, 2000, 3000, 4000, 5000]
labels = ['0-1000', '1001-2000', '2001-3000', '3001-4000', '4001-5000']
df_incremental['Amount_Category'] = pd.cut(df_incremental['Amount'], bins=bins, labels=labels, right=False)

df_incremental.to_csv('transformed_incremental.csv', index=False)

Number of missing values: id              0
Mitai_Sold      0
Date            0
Amount          0
Last_Updated    0
dtype: int64
Number of duplicates: 0


C:\Users\Admin\AppData\Local\Temp\ipykernel_16352\1119926697.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_incremental['Date'] = pd.to_datetime(df_incremental['Date'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_16352\1119926697.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_incremental['Last_Updated'] = pd.to_datetime(df_incremental['Last_Updated'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_16352\1119926697.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 